In [1]:
import sys

sys.path.append("../..")

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
from training.dqnetwork import DQNetwork

2025-10-31 17:12:13.088027: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-31 17:12:13.113359: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-31 17:12:16.005743: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [4]:
from environment.deepqlearning.obstacle_avoidance_env import ObstacleAvoidanceEnv
from utils.reader import get_yaml_path, read_file

In [5]:
server_address = "localhost:50051"
client_name = "RLClient"
env = ObstacleAvoidanceEnv(server_address, client_name)
env.connect_to_client()

2025-10-31 17:12:16,937 — INFO — ✓ Connected to localhost:50051



In [6]:
config_path = get_yaml_path("resources", "configurations", "obstacle-avoidance.yml")
config = read_file(config_path)
# print(config)

In [7]:
env.init(config)

2025-10-31 17:12:16,952 — INFO — ✓ Initialization successful


In [8]:
neuron_count_per_hidden_layer = [64, 32]

In [9]:
episode_count = 50  # Total number of training episodes
episode_max_steps = 2000  # Maximum number of steps per episode

replay_memory_max_size = (
    100000  # Maximum number of transitions stored into the replay memory
)
replay_memory_init_size = (
    1000  # Maximum number of transitions stored into the replay memory
)
batch_size = 64  # Mini-batch size

step_per_update = 4  # Number of total steps executed between successive updates of the action model weights
step_per_update_target_model = 8  # Number of total steps executed between successive replaces of the target model weights

max_epsilon = 1.0  # Exploration probability at start
min_epsilon = 0.01  # Minimum exploration probability
epsilon_decay = 0.0002  # Decay for exploration probability

gamma = 0.99  # Discount factor

moving_avg_window_size = 20  # Number of consecutive episodes to be considered in the calculation of the total reward moving average
moving_avg_stop_thr = 100

In [10]:
from agent.scala_dqagent import DQAgent

agent1 = DQAgent(
    env,
    agent_id="00000000-0000-0000-0000-000000000001",
    action_model=DQNetwork(
        env.observation_space.shape,
        neuron_count_per_hidden_layer,
        env.action_space.n,
        summary=False,
    ),
    target_model=DQNetwork(
        env.observation_space.shape,
        neuron_count_per_hidden_layer,
        env.action_space.n,
        summary=False,
    ),
    epsilon_max=max_epsilon,
    epsilon_min=min_epsilon,
    gamma=gamma,
    replay_memory_max_size=replay_memory_max_size,
    replay_memory_init_size=replay_memory_init_size,
    batch_size=batch_size,
    step_per_update=step_per_update,
    step_per_update_target_model=step_per_update_target_model,
    moving_avg_window_size=moving_avg_window_size,
    moving_avg_stop_thr=moving_avg_stop_thr,
    episode_max_steps=episode_max_steps,
    episodes=episode_count,
)

agents = [agent1]

In [11]:
import time

from training.multi_agent_dqlearning import DQLearning

train_start_time = time.time()

trainer = DQLearning(
    env,
    agents,
    episode_count=episode_count,
    episode_max_steps=episode_max_steps,
)
train_rewards = trainer.simple_dqn_training()

train_finish_time = time.time()
train_elapsed_time = train_finish_time - train_start_time
train_avg_episode_time = train_elapsed_time / episode_count

print(
    f"Train time: {train_elapsed_time / 60.0:.1f}m [{train_avg_episode_time:.1f}s]"
)

<frozen importlib._bootstrap>:488: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.
/nix/store/xfj6pw68yzxq174as0lggbfpqkgxn6z5-python3-3.13.8-env/lib/python3.13/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists
Training DQN:   0%|                                                                                                                                                                                                      | 0/50 [00:00<?, ?ep/s]

2025-10-31 17:12:51,020 — INFO — Episode: 0 | Steps: 2000[2000] | Epsilon: 1.000 | Time: 32.67s | Reward: {'00000000-0000-0000-0000-000000000001': -964.8347416125011} | MovingAvg: {'00000000-0000-0000-0000-000000000001': -964.8347416125011}


Training DQN:   2%|███▊                                                                                                                                                                                          | 1/50 [00:32<26:40, 32.67s/ep]

2025-10-31 17:13:32,301 — INFO — Episode: 1 | Steps: 2000[4000] | Epsilon: 1.000 | Time: 41.28s | Reward: {'00000000-0000-0000-0000-000000000001': -769.2362681213457} | MovingAvg: {'00000000-0000-0000-0000-000000000001': -769.2362681213457}


Training DQN:   4%|███████▌                                                                                                                                                                                      | 2/50 [01:13<30:11, 37.74s/ep]

...

2025-10-31 18:09:22,065 — INFO — Episode: 48 | Steps: 2000[78903] | Epsilon: 0.023 | Time: 105.77s | Reward: {'00000000-0000-0000-0000-000000000001': 762.4032792623693} | MovingAvg: {'00000000-0000-0000-0000-000000000001': 779.037531597297}


Training DQN:  98%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏   | 49/50 [57:03<01:07, 67.26s/ep]

2025-10-31 18:11:02,738 — INFO — Episode: 49 | Steps: 2000[80903] | Epsilon: 0.022 | Time: 100.67s | Reward: {'00000000-0000-0000-0000-000000000001': 1725.7035449906405} | MovingAvg: {'00000000-0000-0000-0000-000000000001': 746.4177591321768}


Training DQN: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [58:44<00:00, 70.49s/ep]

Train time: 58.7m [70.5s]


In [12]:
trainer.play_with_pygame(episodes=5, fps=60)

2025-10-31 18:12:28,972 — INFO — Episode 1/5 - Reward: 1520.3815183927197
2025-10-31 18:12:28,975 — INFO — Episode 2/5 - Reward: 0
2025-10-31 18:12:28,975 — INFO — Episode 3/5 - Reward: 0
2025-10-31 18:12:28,976 — INFO — Episode 4/5 - Reward: 0
2025-10-31 18:12:28,977 — INFO — Episode 5/5 - Reward: 0
